## Indexing and querying the ratebeer dataset using ElasticSearch

In [1]:
import pandas as pd
import json
import requests
import numpy as np
from requests.auth import HTTPBasicAuth
from elasticsearch import Elasticsearch
import elasticsearch
from elasticsearch.helpers import bulk

print(elasticsearch.__version__)

(8, 12, 0)


In [ ]:
# !!! CUSTOMIZE THIS SECTION WITH YOUR CREDENTIALS !!!

USER = 'elastic'
PWD = #confidential
index_name = 'beers'
ES_ENDPOINT = 'https://localhost:9200'

path_to_ca_certificates = 'C:/Users/user/pathtothefolder/elasticsearch-8.12.0/config/certs/http_ca.crt'

### Read data

In [3]:
df_dup = pd.read_csv('../data/ratebeer/ratebeer.csv', index_col = False)
#df = df.set_index('beer_name')
df_dup

,beer_name,beer_style,beer_ABV,review_appearance,review_aroma,review_palate,review_taste,review_overall,review_profileName,review_text
0,Sierra Nevada Bigfoot,Barley Wine,9.6,5,7,4,7,15,Juancho,Draught \tThe big boy from Sierra Nevada. Nice...
1,Icehouse,Pale Lager,5.5,3,2,1,1,1,xav33,"Icehouse\tDeuce-Deuce\tRecent browing, resided..."
2,St Georges Paragon Steam Bitter,California Common,4,3,3,3,6,10,hughie,"Cask at Wellington Arms, Bedford. Gold with a ..."
3,Ayinger Bru-Weisse,German Hefeweizen,5.1,5,8,5,8,15,jredmond,500mL Bottle. 2nd of 13 for the night. Body is...
4,Great Divide Titan IPA,India Pale Ale (IPA),7.1,3,7,4,8,16,gringo332,Bottle. Pours golden/orange with a small white...
...,...,...,...,...,...,...,...,...,...,...
584828,Smiling Moose Mooseberry Wheat,Fruit Beer,-,3,6,3,6,12,Ernest,Draught. Body is hazy medium yellow. Aroma is ...
584829,Deschutes The Dissident,Sour Ale/Wild Ale,10.5,4,8,3,7,15,tokyogoat,poured a burnt ruby orange. Nose was vinegary ...
584830,Stroud Budding,Golden Ale/Blond Ale,4.5,3,6,3,6,12,leaparsons,Bottle. Golden brown with a thin foam. Aroma...
584831,Brakspear Oxford Gold Organic Beer (Bottle),Bitter,4.6,3,4,3,6,11,TEJA,Big but short foam that dies down too a thin l...


### Data Processing

In [4]:
duplicates = df_dup.duplicated().sum()
print("Number of duplicate rows:", duplicates)

Number of duplicate rows: 2784


In [5]:
# Remove duplicates
df = df_dup.drop_duplicates()

In [6]:
# Replace various representations of empty values with None
df.replace(['', ' ', '-', np.nan], None, inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_12880\4141127485.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace(['', ' ', '-', np.nan], None, inplace=True)


In [7]:
# Creating a new column id to index by that
df.reset_index(inplace=True)
df.rename(columns={'index': 'id'}, inplace=True)
df.index.name = "id"
df = df.set_index('id')
df.head()

C:\Users\user\AppData\Local\Temp\ipykernel_12880\1759020812.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'index': 'id'}, inplace=True)


,beer_name,beer_style,beer_ABV,review_appearance,review_aroma,review_palate,review_taste,review_overall,review_profileName,review_text
id,,,,,,,,,,
0,Sierra Nevada Bigfoot,Barley Wine,9.6,5,7,4,7,15,Juancho,Draught \tThe big boy from Sierra Nevada. Nice...
1,Icehouse,Pale Lager,5.5,3,2,1,1,1,xav33,"Icehouse\tDeuce-Deuce\tRecent browing, resided..."
2,St Georges Paragon Steam Bitter,California Common,4,3,3,3,6,10,hughie,"Cask at Wellington Arms, Bedford. Gold with a ..."
3,Ayinger Bru-Weisse,German Hefeweizen,5.1,5,8,5,8,15,jredmond,500mL Bottle. 2nd of 13 for the night. Body is...
4,Great Divide Titan IPA,India Pale Ale (IPA),7.1,3,7,4,8,16,gringo332,Bottle. Pours golden/orange with a small white...


In [8]:
docs = df.to_dict(orient='records')
doc_ids = df.index
print(doc_ids[1])
print(docs[1])

1
{'beer_name': 'Icehouse', 'beer_style': 'Pale Lager', 'beer_ABV': '5.5', 'review_appearance': 3, 'review_aroma': 2, 'review_palate': 1, 'review_taste': 1, 'review_overall': 1, 'review_profileName': 'xav33', 'review_text': 'Icehouse\tDeuce-Deuce\tRecent browing, resided in 11/06\tMedium pour in a pint glass.\t\tPoured a slightly pale yellow with a fizzy mostly diminishing white head.\t\tSmelt of faint grain. \t\tTaste was upfront sweet drainage water with maybe hints of grain flavoring followed by an execrable harsh bitter fat mans bat tub water taste.  Mouthfeel was light bodied, watery-ish, fizzy and strongly astringent on the finish.\t\tOverall, I never actually tasted this beer until today.  Ive drank it a few times and thought it was ok, but never *tasted* it.  Dont taste it.  Just enjoy the buzz.'}


### Indexing with python cURL (Requests)

In [9]:
class Elastic:
    """
    A convenience object to send HTTP requests to Elasticsearch
    """
    def __init__(self, endpoint, username, password, path_to_ca_certificates):
        """
        @param endpoint: the URL of the Elasticsearch instance
        @param username: the Elasticsearch username 
        @param password: the Elasticsearch password
        """
        self.header = {'Content-Type': 'application/json', 'charset':'UTF-8'}
        #self.header={'Content-Type': '--data-binary application/x-ndjson'}
        self.endpoint = endpoint
        self.username = username
        self.password = password
        self.path_to_ca_certificates = path_to_ca_certificates
        self.methods_mapping = {'get': requests.get, 
                                'put':requests.put, 
                                'post':requests.post, 
                                'delete':requests.delete}
        
    def curl(self, method, handle, json=None):
        """
        Sends an HTTP request to the Elasticsearch instance
        @param method: can be 'get', 'put', 'post', 'delete'
        @param handle: the API handle to be appended to the Elasticsearch url
        @param json: the json payload of the HTTP request
        """
        http_method = self.methods_mapping[method.lower()]
        r = http_method(f'{self.endpoint}/{handle}', auth=HTTPBasicAuth(USER, PWD), 
                        headers=self.header, json=json,
                        verify = self.path_to_ca_certificates)
        return r

In [10]:
e = Elastic(ES_ENDPOINT, USER, PWD, path_to_ca_certificates)

In [11]:
create_index_json={
# Point #4 By putting the type text to beer_name and review_text we allow for full-text query
    # while for beer_style we put keyword to omake it useful for filtering creating a DocValue
  "mappings" : {
      "properties" : {
        "beer_name" : {
          "type" : "text" 
        },
        "beer_style" : {
          "type" : "keyword"
        },
        "beer_ABV" : {
          "type" : "float"
        },
        "review_appearance" : {
          "type" : "integer"
        },
        "review_aromaa" : {
          "type" : "integer"
        },
        "review_palate" : {
          "type" : "integer"
        },
        "review_taste" : {
          "type" : "integer"
        },
          "review_overall" : {
          "type" : "integer"
        },
        "review_profileName" : {
          "type" : "text"
        },
        "review_text" : {
          "type" : "text"
        }
      }
  },
  "settings": {
    "refresh_interval": "-1", # Point #2 making a static database I removed the refreshing setting, by putting it to -1 (continued down)
    "number_of_shards": 4, 
    "number_of_replicas": 0, # Point #3  no replicas
    "index.max_result_window": 20000,
    "index" : {
        "similarity" : {
          "default" : {
            "type" : "BM25", "b": 0.825, "k1": 1.2 
    # Point #5 relevance metric is BM25, from default b=0.75 we increase by 10% to 0.825 
          }
        }
    },
    "analysis": {
      "analyzer": {
        "std_english": {"type": "standard", "stopwords": "_english_" }
      }
    }
  }
}

# create an index
r = e.curl('put', index_name, json=create_index_json)
r.json()

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'beers'}

In [12]:
# get the index details and settings
r = e.curl('get', index_name)
r.json()

{'beers': {'aliases': {},
  'mappings': {'properties': {'beer_ABV': {'type': 'float'},
    'beer_name': {'type': 'text'},
    'beer_style': {'type': 'keyword'},
    'review_appearance': {'type': 'integer'},
    'review_aromaa': {'type': 'integer'},
    'review_overall': {'type': 'integer'},
    'review_palate': {'type': 'integer'},
    'review_profileName': {'type': 'text'},
    'review_taste': {'type': 'integer'},
    'review_text': {'type': 'text'}}},
  'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
    'refresh_interval': '-1',
    'number_of_shards': '4',
    'provided_name': 'beers',
    'similarity': {'default': {'type': 'BM25', 'b': '0.825', 'k1': '1.2'}},
    'max_result_window': '20000',
    'creation_date': '1716649304831',
    'analysis': {'analyzer': {'std_english': {'type': 'standard',
       'stopwords': '_english_'}}},
    'number_of_replicas': '0',
    'uuid': 'PLz5g2xTToGU-5JNuX9VcA',
    'version': {'created': '850

In [13]:
# bulk indexing (via official API)

#connect to the local elasticsearch node and authenticate
es = Elasticsearch([ES_ENDPOINT], ca_certs=path_to_ca_certificates, basic_auth=(USER, PWD))

actions = [
  {
    "_index": index_name,
    "_id": doc_id,
    "_source": doc
  }
  for doc_id, doc in list(zip(doc_ids, docs))
]

# Point #1: Taking care of efficient loading by bulk indexing
bulk(es, actions)

(582049, [])

In [14]:
# reset the refresh interval to 2 seconds as it is needed after indexing
r = e.curl('put', 'beers/_settings', {'index' : {'refresh_interval' : '2s'}})
r.json()
# Point #2 (not sure if this would make a rare update but it would make the data base static for sure)
# r = e.curl('put', 'beers/_settings', {'index' : {'refresh_interval' : '-1'}}) 
# r.json()

{'acknowledged': True}

In [15]:
# Checking
r = e.curl('get', f'beers/_doc/{doc_ids[42]}')
r.json()

{'_index': 'beers',
 '_id': '42',
 '_version': 1,
 '_seq_no': 7,
 '_primary_term': 1,
 'found': True,
 '_source': {'beer_name': 'Dogfish Head ApriHop',
  'beer_style': 'Fruit Beer',
  'beer_ABV': '7',
  'review_appearance': 4,
  'review_aroma': 9,
  'review_palate': 4,
  'review_taste': 8,
  'review_overall': 16,
  'review_profileName': 'Jan Primus',
  'review_text': 'Pours a nice, clear copper colour with small white head. Was somewhat skeptical how the apricot and hops would mesh but this turned out just fine as they were both well balanced, and the fruity apricot made the beer very refreshing on a hot evening, adding a subtle touch of sweetness to an ipa. Looking forward to trying this again.'}}

In [16]:
# delete the index
# r = e.curl('delete', 'beers')
# r.json()

### Querying

#### Point #6: Exact match query on the beer name


In [17]:
""" 
execute an exact-match query for the beer "Icehouse"
"""
r1 = e.curl('get', f'beers/_search',{
 "query": {
     "match_phrase": {
         "beer_name": {
             "query": "Icehouse"
}}}})

r1.json()

{'took': 101,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 179, 'relation': 'eq'},
  'max_score': 12.337053,
  'hits': [{'_index': 'beers',
    '_id': '1',
    '_score': 12.337053,
    '_source': {'beer_name': 'Icehouse',
     'beer_style': 'Pale Lager',
     'beer_ABV': '5.5',
     'review_appearance': 3,
     'review_aroma': 2,
     'review_palate': 1,
     'review_taste': 1,
     'review_overall': 1,
     'review_profileName': 'xav33',
     'review_text': 'Icehouse\tDeuce-Deuce\tRecent browing, resided in 11/06\tMedium pour in a pint glass.\t\tPoured a slightly pale yellow with a fizzy mostly diminishing white head.\t\tSmelt of faint grain. \t\tTaste was upfront sweet drainage water with maybe hints of grain flavoring followed by an execrable harsh bitter fat mans bat tub water taste.  Mouthfeel was light bodied, watery-ish, fizzy and strongly astringent on the finish.\t\tOverall, I never actually tasted this

#### Point #7: Full-text query on beer name and review, with review boosted by a factor 1.7 compared to beer name


In [18]:
""" 
execute a full-text query for the query where overall review is 9
"""
query = {
"query": {
    "multi_match": {
            "query": "9",
            "fields": ["beer_name", "review_overall^1.7"],
            "type": "phrase"
}}}
r = e.curl('get', f'beers/_search', query)
r.json()

{'took': 32,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 8.61025,
  'hits': [{'_index': 'beers',
    '_id': '120435',
    '_score': 8.61025,
    '_source': {'beer_name': 'Piraat 9%',
     'beer_style': 'Belgian Strong Ale',
     'beer_ABV': '9',
     'review_appearance': 4,
     'review_aroma': 8,
     'review_palate': 4,
     'review_taste': 8,
     'review_overall': 16,
     'review_profileName': 'CampbellWilson',
     'review_text': 'Bottle from Beers of Europe. Hazy golden yellow with white persistant head. Spicy aroma with orangey hop. Pleasantly warming with an alcohol, spice and hop finish. Good stuff.'}},
   {'_index': 'beers',
    '_id': '177558',
    '_score': 8.61025,
    '_source': {'beer_name': 'Piraat 9%',
     'beer_style': 'Belgian Strong Ale',
     'beer_ABV': '9',
     'review_appearance': 5,
     'review_aroma': 8,
     'review_palate': 5,
     'revie

#### Point #8: Full-text query on the review text, and that filters the results by category

In [19]:
""" 
execute a full-text query for the query "Icehouse" in the review text and filter the results by the beer style
"""
query = {
    "query": {
      "bool": {
       "must": {
         "match": {
                "review_text": "Icehouse"
        }},
        "filter": {
          "term": {
                "beer_style": "Pale Lager"
}}}}}
r = e.curl('get', f'beers/_search', query)
r.json()

{'took': 16,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 34, 'relation': 'eq'},
  'max_score': 13.96607,
  'hits': [{'_index': 'beers',
    '_id': '161519',
    '_score': 13.96607,
    '_source': {'beer_name': 'Icehouse Light',
     'beer_style': 'Pale Lager',
     'beer_ABV': '5',
     'review_appearance': 1,
     'review_aroma': 1,
     'review_palate': 1,
     'review_taste': 1,
     'review_overall': 1,
     'review_profileName': '666seth',
     'review_text': 'Even worse than the regular Icehouse. Stay away! Your better off buying carbonated water (cheaper)'}},
   {'_index': 'beers',
    '_id': '359066',
    '_score': 13.254322,
    '_source': {'beer_name': 'Icehouse',
     'beer_style': 'Pale Lager',
     'beer_ABV': '5.5',
     'review_appearance': 1,
     'review_aroma': 2,
     'review_palate': 1,
     'review_taste': 1,
     'review_overall': 3,
     'review_profileName': 'psychodrama311',
     'revie

#### Point #9: Fuzzy query on the beer name

In [20]:
query ={
  "query": {
    "fuzzy": {
      "beer_name": {
        "value": "Icehouse",
        "fuzziness": "AUTO",
}}}}
r = e.curl('get', f'beers/_search', query)
r.json()

{'took': 41,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 261, 'relation': 'eq'},
  'max_score': 10.794922,
  'hits': [{'_index': 'beers',
    '_id': '1',
    '_score': 10.794922,
    '_source': {'beer_name': 'Icehouse',
     'beer_style': 'Pale Lager',
     'beer_ABV': '5.5',
     'review_appearance': 3,
     'review_aroma': 2,
     'review_palate': 1,
     'review_taste': 1,
     'review_overall': 1,
     'review_profileName': 'xav33',
     'review_text': 'Icehouse\tDeuce-Deuce\tRecent browing, resided in 11/06\tMedium pour in a pint glass.\t\tPoured a slightly pale yellow with a fizzy mostly diminishing white head.\t\tSmelt of faint grain. \t\tTaste was upfront sweet drainage water with maybe hints of grain flavoring followed by an execrable harsh bitter fat mans bat tub water taste.  Mouthfeel was light bodied, watery-ish, fizzy and strongly astringent on the finish.\t\tOverall, I never actually tasted this 

#### Point #10: Full-text query on the review text, with results sorted by decreasing rating on the beer aroma (highest to lowest)

In [21]:
""" 
execute a full-text query for the query "enjoy" in the review text and sorting the results
"""
query = {
    "query": {
      "match": {
            "review_text": "enjoy" 
    }},
    "sort": [{
      "review_aroma": {
            "order": "desc" 
}}]}

r = e.curl('get', f'beers/_search', query)
r.json()

{'took': 43,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 5497, 'relation': 'eq'},
  'max_score': None,
  'hits': [{'_index': 'beers',
    '_id': '11540',
    '_score': None,
    '_source': {'beer_name': 'Presidente',
     'beer_style': 'Pale Lager',
     'beer_ABV': '5',
     'review_appearance': 5,
     'review_aroma': 10,
     'review_palate': 5,
     'review_taste': 10,
     'review_overall': 20,
     'review_profileName': 'rosilet',
     'review_text': 'ligth and flavory, low alcohol volume, its perfect when drunk chilled, maybe thats why some raters did not enjoy it...'},
    'sort': [10]},
   {'_index': 'beers',
    '_id': '53621',
    '_score': None,
    '_source': {'beer_name': 'Founders KBS (Kentucky Breakfast Stout)',
     'beer_style': 'Imperial Stout',
     'beer_ABV': '11.2',
     'review_appearance': 4,
     'review_aroma': 10,
     'review_palate': 5,
     'review_taste': 10,
     'review_overall